Modelling

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import time
import numpy as np

In [ ]:
dataFrame = pd.read_csv('/content/gdrive/MyDrive/iot_23_datasets_small/csv_files/combinedDf.csv',low_memory=False)

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
dataFrame['duration'] = dataFrame['duration'].astype(str)
dataFrame['duration'] = encoder.fit_transform(dataFrame['duration'])
dataFrame['orig_bytes'] = dataFrame['orig_bytes'].astype(str)
dataFrame['orig_bytes'] = encoder.fit_transform(dataFrame['orig_bytes'])
dataFrame['resp_bytes'] = dataFrame['resp_bytes'].astype(str)
dataFrame['resp_bytes'] = encoder.fit_transform(dataFrame['resp_bytes'])
dataFrame['missed_bytes'] = dataFrame['missed_bytes'].astype(str)
dataFrame['missed_bytes'] = encoder.fit_transform(dataFrame['missed_bytes'])
dataFrame['orig_pkts'] = dataFrame['orig_pkts'].astype(str)
dataFrame['orig_pkts'] = encoder.fit_transform(dataFrame['orig_pkts'])
dataFrame['resp_pkts'] = dataFrame['resp_pkts'].astype(str)
dataFrame['resp_pkts'] = encoder.fit_transform(dataFrame['resp_pkts'])
dataFrame['orig_ip_bytes'] = dataFrame['orig_ip_bytes'].astype(str)
dataFrame['orig_ip_bytes'] = encoder.fit_transform(dataFrame['orig_ip_bytes'])
dataFrame['resp_ip_bytes'] = dataFrame['resp_ip_bytes'].astype(str)
dataFrame['resp_ip_bytes'] = encoder.fit_transform(dataFrame['resp_ip_bytes'])
dataFrame['local_orig'] = dataFrame['local_orig'].astype(str)
dataFrame['local_orig'] = encoder.fit_transform(dataFrame['local_orig'])
dataFrame['local_resp'] = dataFrame['local_resp'].astype(str)
dataFrame['local_resp'] = encoder.fit_transform(dataFrame['local_resp'])

In [ ]:
dataFrame.columns

Index(['Unnamed: 0', 'ts', 'id.orig_p', 'id.resp_p', 'proto', 'service',
       'duration', 'orig_bytes', 'resp_bytes', 'conn_state', 'local_orig',
       'local_resp', 'missed_bytes', 'history', 'orig_pkts', 'orig_ip_bytes',
       'resp_pkts', 'resp_ip_bytes', 'label'],
      dtype='object')

In [ ]:
del dataFrame['Unnamed: 0']

In [ ]:
dataFrame['duration']

0            interval
1                   0
2            2.998804
3                   0
4                   0
              ...    
6729222    797.333834
6729223      0.005497
6729224      0.004499
6729225      0.007994
6729226      0.005497
Name: duration, Length: 6729227, dtype: object

In [ ]:
dataFrame.columns

Index(['ts', 'id.orig_p', 'id.resp_p', 'proto', 'service', 'duration',
       'orig_bytes', 'resp_bytes', 'conn_state', 'local_orig', 'local_resp',
       'missed_bytes', 'history', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts',
       'resp_ip_bytes', 'label'],
      dtype='object')

In [ ]:
dataFrame = pd.get_dummies(dataFrame, columns=['proto'])
dataFrame= pd.get_dummies(dataFrame, columns=['conn_state'])
dataFrame.head()

,ts,id.orig_p,id.resp_p,service,duration,orig_bytes,resp_bytes,local_orig,local_resp,missed_bytes,...,conn_state_RSTR,conn_state_RSTRH,conn_state_S0,conn_state_S1,conn_state_S2,conn_state_S3,conn_state_SF,conn_state_SH,conn_state_SHR,conn_state_string
0,time,port,port,string,interval,count,count,bool,bool,count,...,0,0,0,0,0,0,0,0,0,1
1,1525879831.025055,56305,23,-,0,0,0,-,-,0,...,0,0,1,0,0,0,0,0,0,0
2,1525879833.016171,34243,49560,-,2.998804,0,0,-,-,0,...,0,0,1,0,0,0,0,0,0,0
3,1525879834.024847,58525,23,-,0,0,0,-,-,0,...,0,0,1,0,0,0,0,0,0,0
4,1525879838.024838,51524,23,-,0,0,0,-,-,0,...,0,0,1,0,0,0,0,0,0,0


In [ ]:
dataFrame.columns
types = dataFrame.dtypes
print(types)

ts                   object
id.orig_p            object
id.resp_p            object
service              object
duration              int64
orig_bytes            int64
resp_bytes            int64
local_orig            int64
local_resp            int64
missed_bytes          int64
history              object
orig_pkts             int64
orig_ip_bytes         int64
resp_pkts             int64
resp_ip_bytes         int64
label                object
proto_enum            uint8
proto_icmp            uint8
proto_tcp             uint8
proto_udp             uint8
conn_state_OTH        uint8
conn_state_REJ        uint8
conn_state_RSTO       uint8
conn_state_RSTOS0     uint8
conn_state_RSTR       uint8
conn_state_RSTRH      uint8
conn_state_S0         uint8
conn_state_S1         uint8
conn_state_S2         uint8
conn_state_S3         uint8
conn_state_SF         uint8
conn_state_SH         uint8
conn_state_SHR        uint8
conn_state_string     uint8
dtype: object


In [ ]:
X = dataFrame[['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']]
Y = dataFrame['label']

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
normalized_x = scaler.fit_transform(X)
normalized_x

array([[1.        , 1.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.36530337, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.00232979, 0.9996993 , 0.98501071, ..., 1.        , 0.        ,
        0.        ],
       [0.00319455, 0.9996993 , 0.98501071, ..., 1.        , 0.        ,
        0.        ],
       [0.00261268, 0.9996993 , 0.98501071, ..., 1.        , 0.        ,
        0.        ]])

Naive-Bayes Classification

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(normalized_x, Y, random_state=100, test_size=0.2)

In [ ]:
print(len(X_test))
print(len(Y_test))
print(len(X_train))

1345846
1345846
5383381


In [ ]:
df = pd.DataFrame(Y_test)
df.columns
print(df.loc[:,'label'].value_counts())

PartOfAHorizontalPortScan        473777
Benign                           439711
DDoS                             289146
Okiru                            141789
C&C                                 815
Attack                              440
C&C-HeartBeat                       125
C&C-PartOfAHorizontalPortScan        36
String                                3
FileDownload                          2
C&C-Torii                             1
C&C-FileDownload                      1
Name: label, dtype: int64


In [ ]:
clf = GaussianNB()
clf.fit(X_train, Y_train)

GaussianNB()

In [ ]:
y_pred = clf.fit(X_train, Y_train).predict(X_test)
print(y_pred)
print()

clf = GaussianNB().fit(X_train, Y_train)
print()
print(clf.score(X_test, Y_test))
print()



program start...


0.2319388696775114

['C&C-PartOfAHorizontalPortScan' 'C&C-PartOfAHorizontalPortScan'
 'C&C-PartOfAHorizontalPortScan' ... 'C&C-PartOfAHorizontalPortScan'
 'C&C-PartOfAHorizontalPortScan' 'C&C-Torii']

program end...

time cost: 
35.880067586898804 seconds


In [ ]:
print("Classifiction Report :")
print(classification_report(Y_test, y_pred))
from sklearn.metrics import precision_score
precision_score(Y_test, y_pred,zero_division=1, average='macro')

Classifiction Report :


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                               precision    recall  f1-score   support

                       Attack       0.41      0.68      0.51       440
                       Benign       0.98      0.05      0.10    439711
                          C&C       0.35      0.11      0.16       815
             C&C-FileDownload       0.03      1.00      0.05         1
                C&C-HeartBeat       0.09      0.70      0.17       125
C&C-PartOfAHorizontalPortScan       0.00      1.00      0.00        36
                    C&C-Torii       0.00      0.00      0.00         1
                         DDoS       1.00      0.99      1.00    289146
                 FileDownload       0.00      0.50      0.00         2
                        Okiru       0.00      0.00      0.00    141789
    PartOfAHorizontalPortScan       1.00      0.00      0.00    473777
                       String       1.00      1.00      1.00         3

                     accuracy                           0.23   1345846
    

0.48897318387139727

Decision Trees

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
DT = DecisionTreeClassifier()

In [ ]:
X = dataFrame[['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']]
Y = dataFrame['label']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=100, test_size=0.2)

In [ ]:
DT.fit(X_train, Y_train)
print()

print('prediction:')
y_pred = DT.predict(X_test)
print(y_pred)
print()

print('Score:')
score = DT.score(X_test,Y_test)
print(score)

program start...


prediction:
['Okiru' 'PartOfAHorizontalPortScan' 'PartOfAHorizontalPortScan' ...
 'Okiru' 'PartOfAHorizontalPortScan' 'Benign']

Score:
0.9973451643055743
program end...

time cost: 
34.46449041366577 seconds


In [ ]:
print("Classifiction Report :")
print(classification_report(Y_test, y_pred))

Classifiction Report :


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                               precision    recall  f1-score   support

                       Attack       0.98      0.99      0.98       440
                       Benign       1.00      0.99      1.00    439711
                          C&C       0.79      0.35      0.48       815
             C&C-FileDownload       0.00      0.00      0.00         1
                C&C-HeartBeat       0.73      0.58      0.65       125
C&C-PartOfAHorizontalPortScan       0.00      0.00      0.00        36
                    C&C-Torii       0.00      0.00      0.00         1
                         DDoS       1.00      1.00      1.00    289146
                 FileDownload       1.00      0.50      0.67         2
                        Okiru       1.00      1.00      1.00    141789
    PartOfAHorizontalPortScan       0.99      1.00      1.00    473777
                       String       1.00      1.00      1.00         3

                     accuracy                           1.00   1345846
    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import precision_score
precision_score(Y_test, y_pred,zero_division=1, average='macro')

0.7906858844695993